#Aplicar clustering para comprimir Imágenes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.utils import shuffle
from time import time
from PIL import Image
import zipfile
import imageio

In [ ]:
# número de CLUSTERS
n_colors = 64
image_loc = 'content/drive/My Drive/IMG_20191122_171221.jpg'
image_location = '/'+image_loc


# Load the upload photo
imagen_ejemplo = Image.open(image_location)

# Convert to floats instead of the default 8 bits integer coding dividing by 255 
imagen_ejemplo = np.array(imagen_ejemplo, dtype=np.float64) / 255   # IMAGEN ORIGINAL

# Transform Image to a 2D numpy array.
w, h, d = original_shape = tuple(imagen_ejemplo.shape)
assert d == 3
image_array = np.reshape(imagen_ejemplo, (w * h, d))

print("Fitting model on a small sub-sample of the data (3000 pixels)")
t0 = time()
image_array_sample = shuffle(image_array, random_state=0)[:3000]

# Clusterizar los colores 
kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(image_array_sample)
print("done in %0.3fs." % (time() - t0))

# Get labels for all points
print("Predicting color indices on the full image (k-means)")
t0 = time()
labels = kmeans.predict(image_array)  #Asigna el centroide más cercano (predicción) a cada pixel
print("done in %0.3fs." % (time() - t0))

print("Predicting color indices on the full image (random)")
codebook_random = shuffle(image_array, random_state=0)[:n_colors]
t0 = time()
labels_random = pairwise_distances_argmin(codebook_random, image_array, axis=0)
print("done in %0.3fs." % (time() - t0))


def recreate_image(codebook, labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    d = codebook.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = codebook[labels[label_idx]]
            label_idx += 1
    return image


In [ ]:
# Display all results, alongside original image

my_zipfile = zipfile.ZipFile('/content/NewZipfile.zip','w')  #Compresión de la IMAGEN para ver diferencias

plt.figure(1)
plt.clf()
plt.axis('off')
plt.title('Original image')
my_zipfile.write(image_location)
print('Compresión ORIGINAL: ',round(my_zipfile.getinfo(image_loc).file_size/(1024*1024), 2), 'MB' )
plt.imshow(imagen_ejemplo,aspect='auto')


plt.figure(2)
plt.clf()
plt.axis('off')
imagen_clusterizada = recreate_image(kmeans.cluster_centers_, labels, w, h)
imageio.imwrite('/content/compressed_kmeans.jpg', imagen_clusterizada)
my_zipfile.write('/content/compressed_kmeans.jpg')
plt.title('Quantized image (64 colors, K-Means)')
print("Compresión KMEANS en Bytes: ",round(my_zipfile.getinfo('content/compressed_kmeans.jpg').file_size/(1024*1024), 2), 'MB')
plt.imshow(imagen_clusterizada, aspect='auto')


plt.figure(3)
plt.clf()
plt.axis('off')
imagen_recreada=recreate_image(codebook_random, labels_random, w, h)
imageio.imwrite('/content/compressed_random.jpg', imagen_clusterizada)
my_zipfile.write('/content/compressed_random.jpg')
plt.title('Quantized image (64 colors, Random)')
print("Compresión KMEANS en Bytes: ",round(my_zipfile.getinfo('content/compressed_random.jpg').file_size/(1024*1024), 2), 'MB')
plt.imshow(imagen_recreada, aspect='auto')
plt.show()
my_zipfile.close()